#Carregar Bibliotecas


In [19]:
import os
import re
import glob
import sys
import pandas as pd
from datetime import datetime
import numpy as np
from pandas.compat import StringIO

PATH = '/Users/Inmetrics/Documents/Sascar/4 - Coleta/BateriaDeExecucao/postgres/ps41/'
LOGFILE='*out_queries.txt'
#'teste.txt' #' '1403_1008_TESTEout_queries.txt' #'
filesRead = glob.glob(PATH + LOGFILE)  
fileRead = []
df_metricas = pd.DataFrame()
contadorlinha = -1

#Métodos de Trampo sujo

In [20]:
def coletarDataHora(nomeArquivo):
    valor = re.findall("\d\d\d\d\S\d\d\d\d", nomeArquivo)
    date = datetime(2018,int(valor[0][5:7]),int(valor[0][7:9]),int(valor[0][0:2]),int(valor[0][2:4]),0)
    return date

def linhaMais(linha):
    valor = re.findall("^(\s+\S){3}", linha)
    if valor:
        return True
    return False

def processarlinha(linha):
    cond1 = re.findall("^.+FROM  pg_stat_activity", linha)
    cond2 = re.findall("^.+datid", linha)
    cond3 = re.findall("^-----", linha)
    cond4 = re.findall("^.+row", linha)
    cond5 = re.findall("^\s+$", linha)
    if cond1 or cond2 or cond3 or cond4 or cond5:
        return False
    return True

def parse_datetime(x):
    cond = re.findall("^\s+", x)
    if not cond:
        dt = datetime.strptime(x[1:20], '%Y-%m-%d %H:%M:%S')
        return dt
    else:
        return None

def incluirLinha(line, df_metricas):
    #df_metricas = pd.DataFrame()
    line = line.replace('||','+') 
    df = pd.read_csv(StringIO(line), sep="|", names=['datid','datname','pid','usesysid','usename',
                                                     'application_name','client_addr','client_hostname',
                                                     'client_port','backend_start','xact_start',
                                                     'query_start','state_change',
                                                     'wait_event','state','backend_xid','backend_xmin',
                                                     'query', 'Datahora'],
                     converters={'backend_start': parse_datetime,
                                'query_start': parse_datetime,
                                'state_change': parse_datetime})
    df_metricas = pd.concat([df_metricas,df])
    return df_metricas

In [21]:
### REalizar parse
def efetuarParse(arquivo):
    global contadorlinha
    global df_metricas
    
    fileRead = open(arquivo, "r", encoding='utf-8')
    datahora = coletarDataHora(arquivo)
    
    for line in fileRead:
        if (linhaMais(line)):
            line = line.replace('|',' ') 
            df_metricas.iloc[contadorlinha , 17] = str(df_metricas.iloc[contadorlinha , 17]) + line
        else:
            if processarlinha(line):
                df_metricas = incluirLinha(line, df_metricas)
                contadorlinha +=1
                df_metricas.iloc[contadorlinha , 18] =datahora
    fileRead.close()
    return df_metricas
          


In [23]:
for fileLog in filesRead:
    df_metricas = efetuarParse(fileLog)

writer = pd.ExcelWriter('/Users/Inmetrics/Documents/Sascar/Consultas172_19_12_41_out_queries.xlsx' , engine='xlsxwriter')
df_metricas.to_excel(writer, sheet_name='Plan1')
writer.save()